[한국전력공사\_공동주택 전력·기상 융합데이터\_20221231 | 공공데이터포털](https://www.data.go.kr/data/15120804/fileData.do)

한국전력공사_공동주택 전력·기상 융합데이터
한국전력공사와 기상청이 공동주택 부하관리관련 연구개발 활성화를 목적으로 제공하는 데이터셋입니다.
데이터셋은 기상청 동네예보 격자단위 시간대별 공동주택 전력부하와 기상관측값을 포함하고 있습니다.
공동주택 전력부하는 각 연도별 결측치가 없는 단지에 한하여 합계한 통계값이며, 격자내 공동주택이 10개 이상인 격자에 대해서만 제공하고 있습니다.

In [2]:
import os
import pandas as pd

data_folder = 'data'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

dataframes = {}
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    df = pd.read_csv(file_path, encoding="cp949")
    dataframes[csv_file] = df
